In [49]:
import numpy as np
import pandas as pd

from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_selection import RFE


from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV

In [2]:
data = pd.read_csv('data/sub1.csv')

In [3]:
data.head()

,Unnamed: 0,Client_ID,2003,2004,2005,2006,0_M_NO,0_M_YES,0_F_NO,0_F_YES,...,1_F_NO,1_F_YES,var,age,amount,payment,lead,Product,employee,target
0,1,345612161,1,0,0,0,1,0,0,0,...,0,0,1,53,15500.0,Cheque,Walkin,B,A,0
1,2,345612467,1,0,0,0,1,0,0,0,...,0,0,2,45,27744.5,Card,others,B,C,0
2,3,345612552,0,0,1,0,0,0,0,0,...,0,0,2,48,62565.0,Cheque,References,C,B,0
3,4,345612557,0,1,0,0,0,0,0,0,...,0,0,4,64,31918.0,Cheque,Walkin,A,A,0
4,5,345612625,1,0,1,0,1,0,0,0,...,0,0,3,27,59415.5,Card,Walkin,B,A,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417107 entries, 0 to 417106
Data columns (total 22 columns):
Unnamed: 0    417107 non-null int64
Client_ID     417107 non-null int64
2003          417107 non-null int64
2004          417107 non-null int64
2005          417107 non-null int64
2006          417107 non-null int64
0_M_NO        417107 non-null int64
0_M_YES       417107 non-null int64
0_F_NO        417107 non-null int64
0_F_YES       417107 non-null int64
1_M_NO        417107 non-null int64
1_M_YES       417107 non-null int64
1_F_NO        417107 non-null int64
1_F_YES       417107 non-null int64
var           417107 non-null int64
age           417107 non-null float64
amount        417107 non-null float64
payment       417107 non-null object
lead          417107 non-null object
Product       417107 non-null object
employee      417107 non-null object
target        417107 non-null int64
dtypes: float64(2), int64(16), object(4)
memory usage: 73.2+ MB


In [6]:
data_x = data.drop(['Client_ID', 'target', 'Unnamed: 0'], axis = 1)
data_y = data.target

In [27]:
data_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417107 entries, 0 to 417106
Data columns (total 36 columns):
2003                  417107 non-null int64
2004                  417107 non-null int64
2005                  417107 non-null int64
2006                  417107 non-null int64
0_M_NO                417107 non-null int64
0_M_YES               417107 non-null int64
0_F_NO                417107 non-null int64
0_F_YES               417107 non-null int64
1_M_NO                417107 non-null int64
1_M_YES               417107 non-null int64
1_F_NO                417107 non-null int64
1_F_YES               417107 non-null int64
var                   417107 non-null int64
age                   417107 non-null float64
amount                417107 non-null float64
payment_Card          417107 non-null float64
payment_Cash          417107 non-null float64
payment_Cheque        417107 non-null float64
payment_Others        417107 non-null float64
lead_Advertisement    417107 non-null flo

In [26]:
cat_col = ['payment', 'lead', 'Product', 'employee']
data_x = pd.concat([data_x, pd.get_dummies(data_x.ix[:, cat_col])], axis = 1)
data_x.drop(cat_col, axis = 1, inplace=True)

In [28]:
train_x = data_x[data_x['2006'] == 0].values
train_y = data_y[data_x['2006'] == 0].values
test_x = data_x[data_x['2006'] == 1].values
test_y = data_y[data_x['2006'] == 1].values

In [29]:
rmf = RandomForestClassifier(n_jobs=3, n_estimators=100, max_depth=5)

In [30]:
rmf.fit(train_x, train_y);
roc_auc_score(test_y, rmf.predict(test_x))

0.51007556675062971

In [47]:
nb = BernoulliNB(250)

In [40]:
nb.fit(train_x, train_y);
roc_auc_score(test_y, nb.predict(test_x))

0.62554528986235658

In [48]:
for i in range(40, 30, -1):
    rfe = RFE(nb, n_features_to_select=i)
    rfe.fit(train_x, train_y);
    print roc_auc_score(test_y, rfe.predict(test_x)), '\n'

0.712309746919 

0.712309746919 

0.712309746919 

0.712309746919 

0.712309746919 

0.756449414296 

0.756161797632 

0.759278616835 

0.762002996864 

0.737360334308 

